In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import csv
import cv2
import tqdm.notebook as tq
from sklearn.utils import shuffle
from keras.optimizers import *
from keras.losses import *
from keras import Sequential, Model
from keras.layers import *
import PIL
import PIL.Image
import keras.backend as K
#tf.config.run_functions_eagerly(True)

In [44]:
data_location = 'Dataset/'
model_location = 'Models/'
save_model_location = 'saved_model2/'

# Model Generators

In [35]:
def get_discriminator(in_shape):
    
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.00001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def get_generator(in_shape):
    
    n_nodes      = 128 * 15 * 15
    input_image  = Input(in_shape)
    #rescaled    = experimental.preprocessing.Rescaling(1./255)(input_image)
    flat         = Flatten() (input_image)
    fc1          = Dense(n_nodes, kernel_initializer=tf.keras.initializers.HeNormal()) (flat)
    lr1          = ReLU() (fc1)
    out1         = Reshape((15, 15, 128)) (lr1)
    tconv1       = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
                                  kernel_initializer=tf.keras.initializers.HeNormal()) (out1)
    lr2          = LeakyReLU(alpha=0.2) (tconv1)
    noise        = Conv2D(3, (7,7), activation='sigmoid', padding='same',
                          kernel_initializer=tf.keras.initializers.HeNormal()) (lr2)
    
    #noise = tf.clip_by_value(noise, clip_value_min=-0.15, clip_value_max=0.15)
    output_image = 0.9 * input_image +  0.1 *noise 
    output_image = tf.clip_by_value(output_image, clip_value_min=0, clip_value_max=1)
    
    model = Model(inputs=input_image, outputs=output_image)
    return model

def custom_loss(y_actual, y_predicted):
    loss = K.categorical_crossentropy(y_actual, y_predicted)
    return -1 * loss


class GAN(tf.keras.Model):

    def __init__(self, generator, discriminator, classifier):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.classifier = classifier
        self.discriminator.trainable = False
        self.classifier.trainable = False

    def call(self, input_sign):
        noisy_sign = self.generator(input_sign)
        gan_out = self.discriminator(noisy_sign)
        adv_out = self.classifier(noisy_sign)
        return gan_out, adv_out



def define_gan(generator, discriminator, classifier):   # Discriminator Loss Model
    
    model = GAN(generator, discriminator, classifier)
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile( loss={'output_1': 'binary_crossentropy', 
                         'output_2': custom_loss},
                  optimizer=opt,
                  loss_weights=[0.1, 0.6])
    return model


#  Training Function

In [ ]:
def train_all_models(d_model, g_model, gan_model, train_ds, num_epochs = 10):
    N_runs = len(train_ds)
    for i in range(num_epochs):
        print('Epoch : ', i)
        Dloss, Dacc, GD_loss, GAN_loss = 0,0,0,0
        for n in tq.tqdm( range(N_runs) ):
            
            X_dis, Y_dis = generate_discriminator_dataset(g_model, train_ds, batch_size)
            if(n > 0):
                dloss, dacc  = d_model.train_on_batch(X_dis, Y_dis)
            else:
                dloss, dacc = d_model.evaluate(X_dis, Y_dis, verbose = 0)
            
            X_gan, Y_gan, Y_adv = generate_gan_dataset(train_ds)
            ganloss      = gan_model.train_on_batch(X_gan, {'output_1' : Y_gan, 'output_2' : Y_adv})
            
            
#             Dloss = Dloss + dloss
#             Dacc = Dacc + dacc
#             GAN_loss = GD_loss + ganloss[1]
#             ADV_loss = GAN_loss + ganloss[2]
#             Future Work : Joint Trainng Model"
            print('Discriminator Loss : ', round(dloss, 2), '  Accuracy', round(dacc,2),
                  ' || GAN loss: ', ganloss)
#         print('Discriminator Loss : ', round(Dloss/N_runs, 2), '  Accuracy', round(Dacc/N_runs,2),
#               '|| GD Loss: ', round(GD_loss/N_runs,2) , ' || GAN loss: ', round(GAN_loss/N_runs,2))
        
        if( i%20 == 0 ):
            g_model.save(save_model_location +'gen_'+ str(i) + '.h5')

# DataBatch Generator

In [8]:
def generate_real_datas(train_ds):
    image_batch, label_batch = next(iter(train_ds))
    return image_batch, np.ones((len(image_batch), 1))

def generate_fake_datas(g_model, train_ds):
    image_batch, label_batch = next(iter(train_ds))
    fake_images = g_model.predict(image_batch)
    return fake_images, np.zeros( (len(fake_images), 1) )

def generate_discriminator_dataset(g_model, train_ds, batch_size):
    X_real, Y_real = generate_real_datas(train_ds)
    X_fake, Y_fake = generate_fake_datas(g_model, train_ds)
    X = np.concatenate(( X_real, X_fake ))
    Y = np.concatenate(( Y_real, Y_fake ))
    return X,Y

def generate_gan_dataset(train_ds):
    image_batch, label_batch = next(iter(train_ds))
    return image_batch, np.ones((len(image_batch), 1)), tf.one_hot(label_batch, 4)

#  Execution 

In [36]:
sign_shape = (30, 30, 3)
num_classes = 43
num_epochs = 30

batch_size = 32
img_height = 30
img_width = 30
#images, labels = readTrafficSigns(data_location, 43)
#images = images/256

import pathlib
data_dir = pathlib.Path('Dataset/Signs/')
train_ds = tf.keras.preprocessing.image_dataset_from_directory( data_dir,  validation_split=0.2,
  subset="training",  seed=123,  image_size=(img_height, img_width),  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(  data_dir,  validation_split=0.2,
  subset="validation",  seed=123,  image_size=(img_height, img_width),  batch_size=batch_size)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

print(" Loaded training data in train_ds and validation data in val_ds")

Found 700 files belonging to 3 classes.
Using 560 files for training.
Found 700 files belonging to 3 classes.
Using 140 files for validation.
 Loaded training data in train_ds and validation data in val_ds


In [38]:
# Create the models Discriminator, Generator and GAN
discriminator = get_discriminator(sign_shape)
generator = get_generator(sign_shape)
classifier = tf.keras.models.load_model(model_location +'model.h5')
gan_model = define_gan(generator, discriminator, classifier)

In [39]:
# Train Functions
train_all_models(discriminator, generator, gan_model, train_ds, num_epochs=40)

Epoch :  0


Discriminator Loss :  0.7   Accuracy 0.47  || GAN loss:  [-0.13354915380477905, 0.6404645442962646, -0.32932600378990173]
Discriminator Loss :  0.7   Accuracy 0.41  || GAN loss:  [-0.0547293946146965, 0.6586251258850098, -0.20098650455474854]
Discriminator Loss :  0.7   Accuracy 0.45  || GAN loss:  [0.02638263627886772, 0.6581035852432251, -0.06571286916732788]
Discriminator Loss :  0.71   Accuracy 0.39  || GAN loss:  [-0.2924259901046753, 0.6586306095123291, -0.5971484184265137]
Discriminator Loss :  0.7   Accuracy 0.36  || GAN loss:  [-0.20751847326755524, 0.6494988203048706, -0.4541139006614685]
Discriminator Loss :  0.71   Accuracy 0.41  || GAN loss:  [-0.6731193661689758, 0.6673593521118164, -1.2330920696258545]
Discriminator Loss :  0.7   Accuracy 0.52  || GAN loss:  [-0.5604673027992249, 0.6603871583938599, -1.044176697731018]
Discriminator Loss :  0.7   Accuracy 0.42  || GAN loss:  [-0.2411663979291916, 0.6632359027862549, -0.5124832987785339]
Discriminator Loss :  0.68   Accur

Discriminator Loss :  0.69   Accuracy 0.47  || GAN loss:  [-1.242165446281433, 0.6720178127288818, -2.182278633117676]
Discriminator Loss :  0.7   Accuracy 0.47  || GAN loss:  [-1.573931336402893, 0.6822885274887085, -2.736933469772339]
Discriminator Loss :  0.71   Accuracy 0.48  || GAN loss:  [-1.3232215642929077, 0.6561684608459473, -2.314730644226074]
Discriminator Loss :  0.71   Accuracy 0.5  || GAN loss:  [-1.6942524909973145, 0.6692718267440796, -2.9352993965148926]
Discriminator Loss :  0.71   Accuracy 0.36  || GAN loss:  [-1.0590393543243408, 0.6655689477920532, -1.8759937286376953]
Discriminator Loss :  0.71   Accuracy 0.48  || GAN loss:  [-1.9111469984054565, 0.674573540687561, -3.2976737022399902]
Discriminator Loss :  0.7   Accuracy 0.44  || GAN loss:  [-1.7145295143127441, 0.6712847948074341, -2.9694297313690186]
Discriminator Loss :  0.7   Accuracy 0.47  || GAN loss:  [-2.1435296535491943, 0.6798790693283081, -3.6858625411987305]
Discriminator Loss :  0.71   Accuracy 0.48

Discriminator Loss :  0.7   Accuracy 0.42  || GAN loss:  [-2.8713481426239014, 0.6813685297966003, -4.899141788482666]
Discriminator Loss :  0.7   Accuracy 0.44  || GAN loss:  [-4.465841770172119, 0.6858196258544922, -7.557373046875]
Discriminator Loss :  0.69   Accuracy 0.55  || GAN loss:  [-4.1905646324157715, 0.6777639389038086, -7.097234725952148]
Discriminator Loss :  0.68   Accuracy 0.58  || GAN loss:  [-1.6518144607543945, 0.6837891340255737, -2.8669888973236084]
Discriminator Loss :  0.69   Accuracy 0.47  || GAN loss:  [-2.407834768295288, 0.6832913160324097, -4.12693977355957]
Discriminator Loss :  0.71   Accuracy 0.45  || GAN loss:  [-3.7638731002807617, 0.6849467754364014, -6.387279510498047]
Discriminator Loss :  0.7   Accuracy 0.5  || GAN loss:  [-2.546583890914917, 0.6995395421981812, -4.360896110534668]
Discriminator Loss :  0.69   Accuracy 0.48  || GAN loss:  [-2.6664979457855225, 0.6614159941673279, -4.554399013519287]
Discriminator Loss :  0.69   Accuracy 0.53  || GAN

Discriminator Loss :  0.69   Accuracy 0.53  || GAN loss:  [-4.008339881896973, 0.7116051912307739, -6.799166679382324]
Discriminator Loss :  0.69   Accuracy 0.53  || GAN loss:  [-3.5007967948913574, 0.6863959431648254, -5.949060440063477]
Discriminator Loss :  0.7   Accuracy 0.5  || GAN loss:  [-4.027894020080566, 0.6838460564613342, -6.8271307945251465]
Discriminator Loss :  0.7   Accuracy 0.5  || GAN loss:  [-3.933436393737793, 0.6958619952201843, -6.671703815460205]
Discriminator Loss :  0.69   Accuracy 0.5  || GAN loss:  [-4.561773777008057, 0.6888185143470764, -7.717759609222412]
Discriminator Loss :  0.69   Accuracy 0.52  || GAN loss:  [-4.357302665710449, 0.6953046321868896, -7.378055095672607]
Discriminator Loss :  0.68   Accuracy 0.53  || GAN loss:  [-3.310058832168579, 0.7015304565429688, -5.633686065673828]
Discriminator Loss :  0.69   Accuracy 0.59  || GAN loss:  [-2.643275260925293, 0.6825723648071289, -4.51922082901001]
Discriminator Loss :  0.69   Accuracy 0.45  || GAN l

...

Discriminator Loss :  0.67   Accuracy 0.75  || GAN loss:  [-9.599188804626465, 0.7448376417160034, -16.122787475585938]
Discriminator Loss :  0.67   Accuracy 0.73  || GAN loss:  [-4.862611293792725, 0.7266623973846436, -8.225461959838867]
Discriminator Loss :  0.66   Accuracy 0.77  || GAN loss:  [-9.542038917541504, 0.7562424540519714, -16.029438018798828]
Discriminator Loss :  0.67   Accuracy 0.75  || GAN loss:  [-10.203433990478516, 0.7687259912490845, -17.13384437561035]
Discriminator Loss :  0.67   Accuracy 0.69  || GAN loss:  [-11.444178581237793, 0.7418199777603149, -19.197265625]
Discriminator Loss :  0.69   Accuracy 0.69  || GAN loss:  [-10.314042091369629, 0.7677909135818481, -17.318035125732422]
Discriminator Loss :  0.68   Accuracy 0.67  || GAN loss:  [-9.055548667907715, 0.7632976174354553, -15.219797134399414]
Discriminator Loss :  0.66   Accuracy 0.75  || GAN loss:  [-15.205299377441406, 0.7698999047279358, -25.470481872558594]
Discriminator Loss :  0.67   Accuracy 0.69  

Discriminator Loss :  0.67   Accuracy 0.69  || GAN loss:  [-11.952718734741211, 0.7465635538101196, -20.045623779296875]
Discriminator Loss :  0.67   Accuracy 0.66  || GAN loss:  [-9.949039459228516, 0.7546994686126709, -16.707515716552734]
Discriminator Loss :  0.68   Accuracy 0.62  || GAN loss:  [-11.940437316894531, 0.7501508593559265, -20.025753021240234]
Discriminator Loss :  0.68   Accuracy 0.69  || GAN loss:  [-8.53573989868164, 0.7686030864715576, -14.354333877563477]
Discriminator Loss :  0.66   Accuracy 0.64  || GAN loss:  [-13.989361763000488, 0.7482775449752808, -23.44031524658203]
Discriminator Loss :  0.67   Accuracy 0.61  || GAN loss:  [-12.25932788848877, 0.7432205677032471, -20.556081771850586]
Discriminator Loss :  0.68   Accuracy 0.58  || GAN loss:  [-9.347099304199219, 0.7493371963500977, -15.703388214111328]
Discriminator Loss :  0.67   Accuracy 0.67  || GAN loss:  [-11.609668731689453, 0.7371875643730164, -19.472312927246094]
Discriminator Loss :  0.67   Accuracy 

Discriminator Loss :  0.67   Accuracy 0.77  || GAN loss:  [-8.987921714782715, 0.7473713159561157, -15.10443115234375]
Discriminator Loss :  0.66   Accuracy 0.73  || GAN loss:  [-10.299925804138184, 0.7520061135292053, -17.29187774658203]
Discriminator Loss :  0.68   Accuracy 0.64  || GAN loss:  [-11.792701721191406, 0.7360233664512634, -19.77717399597168]
Discriminator Loss :  0.67   Accuracy 0.61  || GAN loss:  [-10.239553451538086, 0.7544523477554321, -17.19166374206543]
Discriminator Loss :  0.68   Accuracy 0.66  || GAN loss:  [-11.837108612060547, 0.7648427486419678, -19.855987548828125]
Discriminator Loss :  0.67   Accuracy 0.69  || GAN loss:  [-12.782781600952148, 0.7583931684494019, -21.431034088134766]
Discriminator Loss :  0.69   Accuracy 0.58  || GAN loss:  [-12.827116966247559, 0.7802155017852783, -21.508563995361328]
Discriminator Loss :  0.67   Accuracy 0.64  || GAN loss:  [-6.976632595062256, 0.749494731426239, -11.752635955810547]
Discriminator Loss :  0.67   Accuracy 0

Discriminator Loss :  0.67   Accuracy 0.75  || GAN loss:  [-12.361416816711426, 0.7628403902053833, -20.729501724243164]
Discriminator Loss :  0.66   Accuracy 0.69  || GAN loss:  [-10.918566703796387, 0.7459313869476318, -18.32193374633789]
Discriminator Loss :  0.67   Accuracy 0.75  || GAN loss:  [-12.654417991638184, 0.7560412883758545, -21.216703414916992]
Discriminator Loss :  0.67   Accuracy 0.67  || GAN loss:  [-8.572577476501465, 0.7819613814353943, -14.41795539855957]
Discriminator Loss :  0.66   Accuracy 0.72  || GAN loss:  [-9.980829238891602, 0.7517086267471313, -16.759998321533203]
Discriminator Loss :  0.67   Accuracy 0.72  || GAN loss:  [-12.530984878540039, 0.7667622566223145, -21.012767791748047]
Discriminator Loss :  0.68   Accuracy 0.66  || GAN loss:  [-8.081869125366211, 0.7787004709243774, -13.599565505981445]
Discriminator Loss :  0.68   Accuracy 0.69  || GAN loss:  [-14.576493263244629, 0.7812228202819824, -24.424358367919922]
Discriminator Loss :  0.68   Accuracy

Discriminator Loss :  0.66   Accuracy 0.81  || GAN loss:  [-12.705636978149414, 0.7580718994140625, -21.302406311035156]
Discriminator Loss :  0.67   Accuracy 0.73  || GAN loss:  [-8.38852310180664, 0.7668224573135376, -14.108675003051758]
Discriminator Loss :  0.67   Accuracy 0.69  || GAN loss:  [-11.941110610961914, 0.7594279050827026, -20.02842140197754]
Discriminator Loss :  0.68   Accuracy 0.64  || GAN loss:  [-8.758307456970215, 0.7613128423690796, -14.7240629196167]
Discriminator Loss :  0.68   Accuracy 0.67  || GAN loss:  [-10.2587251663208, 0.7776573300361633, -17.22748374938965]
Discriminator Loss :  0.68   Accuracy 0.7  || GAN loss:  [-8.075489044189453, 0.757064700126648, -13.585325241088867]
Discriminator Loss :  0.69   Accuracy 0.55  || GAN loss:  [-10.868253707885742, 0.7407060861587524, -18.237207412719727]
Discriminator Loss :  0.68   Accuracy 0.67  || GAN loss:  [-11.282890319824219, 0.7385785579681396, -18.927913665771484]
Discriminator Loss :  0.66   Accuracy 0.67  

# Load and Run the Models

In [32]:
load = 0
if(load = 1):
    generator = tf.keras.models.load_model('saved_model2/gen_60.h5')

In [46]:
#Training Missclassification
count = 0
total_count = 0
for x in tq.tqdm(train_ds):
    for y in range(len(x[0])):
        if( x[1][y] != 3):
            im = tf.expand_dims(x[0][y], 0)
            predb = generator(im )
            pred = classifier.predict(predb)
            
            total_count = total_count + 1
            count = count + (np.argmax(pred[0]) != x[1][y].numpy())
            #print(np.argmax(pred[0]) , x[1][y].numpy())
count, total_count

(394, 560)

In [47]:
# Validation Missclassification
count = 0
total_count = 0
for x in tq.tqdm(val_ds):
    for y in range(len(x[0])):
        if( x[1][y] != 3):
            im = tf.expand_dims(x[0][y], 0)
            predb = generator(im )
            pred = classifier.predict(predb)
            
            total_count = total_count + 1
            count = count + (np.argmax(pred[0]) != x[1][y].numpy())
            #print(np.argmax(pred[0]) , x[1][y].numpy())
count, total_count

(92, 140)